In [1]:
import pandas as pd
import datetime as dt
import pandasql as ps
import os
import zipfile as zp
from datetime import datetime
import matplotlib.pyplot as plt
import plotly.express as px

#from kaggle.api.kaggle_api_extended import KaggleApi
#api = KaggleApi()
#api.authenticate()

#dataset_name=api.dataset_list_files('prakrutchauhan/indian-candidates-for-general-election-2019').files

DIR=r'/home/anuj/Downloads/'
ZIp='archive(2).zip'

os.chdir(DIR)
zf = zp.ZipFile(f'{ZIp}')

dfs={}
for name in zf.namelist():
    df_name= name.replace('.csv','')
    dfs[df_name]=pd.read_csv(zf.open(name,mode='r'),encoding='latin')

df=dfs['GOI']
df.columns=df.columns.str.replace(' ','_')
df.columns=df.columns.str.replace('\n','_')
df.columns=df.columns.str.replace('(','')
df.columns=df.columns.str.replace(')','')
df.columns=df.columns.str.replace('-','')
df.columns=df.columns.str.replace('/','')

df=df[df['Country_States_Union_Territories_Name']!='INDIA']

#os.chdir(r'C:\Users\anuj\Documents\My Tableau Repository\Datasources')
#df.to_csv('indian_elections.csv',index=False)

/tmp/ipykernel_19559/2890825634.py:30: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns=df.columns.str.replace('(','')
/tmp/ipykernel_19559/2890825634.py:31: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df.columns=df.columns.str.replace(')','')


In [38]:
df2=df.melt(id_vars=['Category','Country_States_Union_Territories_Name'],value_name='Literacy_Rate',var_name='Type')
df2=df2.rename(columns={"Country_States_Union_Territories_Name":"STATE_UT"})
df2['Year']=df2['Type'].apply(lambda x: x.split('__')[2])
df2['Typ']=df2['Type'].apply(lambda x: x.split('__')[1])
del df2['Type']


In [3]:
df2.isnull().isnull().sum()
df2[df2.duplicated()]

,Category,STATE_UT,Literacy_Rate,Year,Typ


In [4]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 210 entries, 0 to 209
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Category       210 non-null    object 
 1   STATE_UT       210 non-null    object 
 2   Literacy_Rate  210 non-null    float64
 3   Year           210 non-null    object 
 4   Typ            210 non-null    object 
dtypes: float64(1), object(4)
memory usage: 8.3+ KB


In [5]:
# NUMBER OF UNIQUE VALUES

for col in df2.columns:
    print(col,df2[col].nunique())

Category 2
STATE_UT 35
Literacy_Rate 163
Year 2
Typ 3


### AVG LITERACY RATE BY STATE

In [99]:
q1=df2.query("Typ!='Total'").groupby(['STATE_UT','Year','Typ']).agg({"Literacy_Rate":"mean"}).reset_index().sort_values(by=['Year','Literacy_Rate'],ascending=[False,True])
q1['Literacy_Rate_lab']=q1['Literacy_Rate'].apply(lambda x: "{:.1%}".format(x/100))
fig=px.bar(q1,x='Literacy_Rate',y='STATE_UT',orientation='h',facet_col='Year',color='Typ',hover_data=q1.columns,text='Literacy_Rate_lab',height=1000)
fig.update_layout({"showlegend":True,"title":"Literacy Rate by State","title_font_size":25,"font_family":"Courier New","title_font_color":"red","legend_title_font_color":"green"})
fig.write_image(format='png',file='Literacy Rate by State.png',validate=True)
fig.show()

In [48]:
q2=df2.groupby(['STATE_UT','Year','Typ']).agg({"Literacy_Rate":"mean"}).sort_values(by='Literacy_Rate',ascending=True).reset_index()
q2=q2.set_index(['STATE_UT','Year','Typ'])['Literacy_Rate'].unstack().reset_index().rename(columns={"2001":"last_census","2011":"latest_census"})
q2['chng']=q2['latest_census']-q2['last_census']
q2=q2.sort_values(by='chng',ascending=True)
q2['chng_lab']=q2['chng'].apply(lambda x: "{:.1%}".format(x/100))


fig=px.bar(q2,x='chng',y='STATE_UT',orientation='h',hover_data=q2.columns,text='chng_lab',height=1000)
fig.update_layout({"title":"States with change in Literacy Rate","title_font_size":25,"font_family":"Courier New","title_font_color":"red","legend_title_font_color":"green"})
fig.write_image(format='png',file='States with change in Literacy Rate.png',validate=True)
fig.show()


### Difference b/w Literacy Rates Max and Minimum

In [92]:
q3=df2.groupby(['STATE_UT','Year']).agg({"Literacy_Rate":"mean"}).sort_values(by='Literacy_Rate',ascending=True).reset_index()
q3=q3.set_index(['STATE_UT','Year'])['Literacy_Rate'].unstack().reset_index().rename(columns={"2001":"last_census","2011":"latest_census"})
q3.groupby('STATE_UT').agg({"latest_census":"mean"}).rank(ascending=False)
q3=pd.concat([q3,q3.groupby('STATE_UT').agg({"latest_census":"mean"}).rank(ascending=False).reset_index().rename(columns={"latest_census":"LR_Rank","STATE_UT":"STATE"})],axis=1)[['STATE_UT','latest_census','LR_Rank']]

ans=q3.sort_values(by='LR_Rank',ascending=True).head(1).append(q3.sort_values(by='LR_Rank',ascending=True).tail(1))
ans[['latest_census']].reset_index(drop=True).diff()

Year,latest_census
0,NaN
1,-27.866667


In [110]:
q4=df2[(df2['Year']=='2011') & (df2['Typ']!='Total')].groupby(['STATE_UT','Typ']).agg({"Literacy_Rate":"mean"}).sort_values(by='Literacy_Rate',ascending=True).reset_index()
q4=q4.set_index(['STATE_UT','Typ'])['Literacy_Rate'].unstack().reset_index()
q4

Typ,STATE_UT,Rural,Urban
0,A & N Islands,84.5,90.1
1,Andhra Pradesh,60.4,80.1
2,Arunachal Pradesh,59.9,82.9
3,Assam,69.3,88.5
4,Bihar,59.8,76.9
5,Chandigarh,80.7,86.2
6,Chhattisgarh,66.0,84.0
7,D & N Haveli,64.1,89.8
8,Daman & Diu,81.4,89.0
9,Goa,86.6,90.0
